In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import numpy as np
import cv2
from PIL import Image

import torch
from torch.autograd import Variable
from torch import nn
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torchvision.datasets import OxfordIIITPet
from torchvision import transforms
import torchvision as tv
import torch.nn.functional as F
from torch import Tensor

In [2]:
transform = transforms.Compose(
    [
        transforms.Resize((256, 256)),
        transforms.RandomHorizontalFlip(0.3),
        transforms.RandomAffine(
            degrees=(-3, 3), translate=(0.05, 0.05),
            interpolation=tv.transforms.InterpolationMode.BILINEAR
        ),
        transforms.ToTensor(), 
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ]
)

train_ds = OxfordIIITPet(
    root = "~/data", split = "trainval", target_types = "category", transform = transform, download = True
)

test_ds = OxfordIIITPet(
    root = "~/data", split = "test", target_types = "category", transform = transform, download = True
)

extra_train_ds, valid_ds, test_ds = random_split(test_ds, [0.8, 0.1, 0.1])

train_ds = ConcatDataset([train_ds, extra_train_ds])

train_dl = DataLoader(train_ds, batch_size=64, shuffle=True, pin_memory = True, num_workers=24)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=True, pin_memory = True, num_workers=24)
test_dl = DataLoader(test_ds, batch_size=64, shuffle=True, pin_memory = True, num_workers=24)

print("#Training Samples: {}".format(len(train_ds)))
print("#Validating Samples: {}".format(len(valid_ds)))
print("#Testing Samples: {}".format(len(test_ds)))

print("#Training Batch: {}".format(len(train_dl)))
print("#Validating Batch: {}".format(len(valid_dl)))
print("#Testing Batch: {}".format(len(test_dl)))

print("# Class: {}".format(len(train_ds.datasets[0].class_to_idx)))

#Training Samples: 6616
#Validating Samples: 367
#Testing Samples: 366
#Training Batch: 104
#Validating Batch: 6
#Testing Batch: 6
# Class: 37


In [3]:
sample_data, sample_cate = train_ds[100]

print(f"data shape: {sample_data.shape}")
print(f"cate data/type: {sample_cate}/{type(sample_cate)}")

data shape: torch.Size([3, 256, 256])
cate data/type: 2/<class 'int'>


In [4]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

In [5]:
class Down(nn.Module):

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [6]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        self.inc = (DoubleConv(n_channels, 64))
        self.down1 = (Down(64, 128)) #64, 128
        self.down2 = (Down(128, 256)) #128, 256
        self.down3 = (Down(256, 512)) #256, 512
        self.down4 = (Down(512, 1024))
        self.down5 = (Down(1024, 2048))
        
        self.classifier = nn.Sequential(
            nn.Linear(2048, 256),
            nn.ReLU(),
            nn.Linear(256, self.n_classes)
        )

    def forward(self, x):
        x = self.inc(x)
        x = self.down1(x)
        x = self.down2(x)
        x = self.down3(x)
        x = self.down4(x)
        x = self.down5(x)        
        logits = self.classifier(x.mean(dim=(2, 3)))
        return logits

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu", index = 1)

model = UNet(3, 37).to(device)

optimizer = Adam(params = model.parameters(), lr = 0.001)

epochs = 100

scheduler = CosineAnnealingLR(optimizer, epochs * len(train_dl))

loss_fn = nn.CrossEntropyLoss()

In [8]:
old_loss = 1e26
best_dct = None
last_dst = None
for epoch in range(epochs):
    model.train()
    tr_total_loss = 0
    tr_total_corr = 0 
    for train_img, train_cate in tqdm(train_dl):
        train_img = train_img.to(device)
        train_cate = train_cate.to(device)

        train_logits = model(train_img)
        train_loss = loss_fn(train_logits, train_cate)

        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()

        scheduler.step()

        tr_total_loss += train_loss.item()
        tr_total_corr += (train_logits.argmax(dim=1) == train_cate).sum().item()

    model.eval()
    with torch.no_grad():
        va_total_loss = 0
        va_total_corr = 0
        for valid_img, valid_cate in tqdm(valid_dl):
            valid_img = valid_img.to(device)
            valid_cate = valid_cate.to(device)
            
            valid_logits = model(valid_img)
            valid_loss = loss_fn(valid_logits, valid_cate)

            va_total_loss += valid_loss.item()
            va_total_corr += (valid_logits.argmax(dim=1) == valid_cate).sum().item()
            
    mean_train_loss = tr_total_loss/len(train_dl)
    mean_valid_loss = va_total_loss/len(test_dl)

    if mean_valid_loss <= old_loss:
        old_loss = mean_valid_loss
        best_dct = model.state_dict()
    
    last_dct = model.state_dict()

    mean_train_corr = tr_total_corr/len(train_ds)
    mean_valid_corr = va_total_corr/len(test_ds)

    print(f"Epoch: {epoch} - TrainLoss: {mean_train_loss} - ValidLoss: {mean_valid_loss}")
    print(f"Epoch: {epoch} - TrainACC: {mean_train_corr} - ValidACC: {mean_valid_corr}")
model.load_state_dict(best_dct)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.93it/s]


Epoch: 0 - TrainLoss: 3.5708817404050093 - ValidLoss: 3.4898411432902017
Epoch: 0 - TrainACC: 0.0471584038694075 - ValidACC: 0.05737704918032787


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.98it/s]


Epoch: 1 - TrainLoss: 3.428968858260375 - ValidLoss: 3.3760916789372764
Epoch: 1 - TrainACC: 0.07240024183796856 - ValidACC: 0.07923497267759563


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.97it/s]


...

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.88it/s]


Epoch: 98 - TrainLoss: 0.0029729183941246727 - ValidLoss: 1.2432270050048828
Epoch: 98 - TrainACC: 1.0 - ValidACC: 0.7786885245901639


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.96it/s]

Epoch: 99 - TrainLoss: 0.0030003827000305485 - ValidLoss: 1.148699591557185
Epoch: 99 - TrainACC: 0.999546553808948 - ValidACC: 0.7786885245901639


<All keys matched successfully>

In [9]:
model.eval()
with torch.no_grad():
    ts_total_loss = 0
    ts_total_corr = 0
    for test_img, test_cate in tqdm(test_dl):
        test_img = test_img.to(device)
        test_cate = test_cate.to(device)
        
        test_logits = model(test_img)
        test_loss = loss_fn(test_logits, test_cate)

        ts_total_loss += test_loss.cpu().item()
        ts_total_corr += (test_logits.argmax(dim=1) == test_cate).sum().item()

mean_test_loss = ts_total_loss/len(test_dl)
mean_test_corr = ts_total_corr/len(test_ds)

print(f"TestLoss: {mean_test_loss} - TestACC: {mean_test_corr}")

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 6/6 [00:02<00:00,  2.81it/s]

TestLoss: 1.3638700445493062 - TestACC: 0.7404371584699454
